In [1]:
from glob import glob
import pandas as pd
import numpy as np
from Bio import PDB
import os
from rdkit.Chem.rdmolfiles import MolFromSmiles, MolToSmiles

In [2]:
info_path = os.path.abspath("datasets/p2rank-datasets/")
pdb_path = os.path.abspath("datasets/p2rank-datasets/holo4k")
pdb_paths = os.path.abspath("datasets/p2rank-datasets/holo4k")

In [3]:
complex_list = os.listdir(pdb_path)

In [4]:
import os
os.environ["PATH"] += os.pathsep + "/opt/conda/envs/team05/bin"
os.environ["PATH"] += os.pathsep + "/workspace/chimera/bin"

In [5]:
#%%bash -s $pdb_path

#path=$1

#for file in $path/*/*'.pdb';do
#    output_path=${file%.pdb}"_chimera.pdb"
#    echo -e "open $file \n write format pdb 0 $output_path \n stop" | chimera --nogui
#done > chimer.log

In [6]:
def read_file(file):
    return file.readlines()

def preprocessing_lig_code(lines):
    lig_info = dict()
    for line in lines:
        line = line.strip()
        if not line or line.startswith("HEADER") or line.startswith("#"):
            continue

        parts = line.split()
        if len(parts) < 2:
            continue  # 불완전한 줄 무시

        try:
            pdb_id = parts[0].split("/")[1].split(".")[0]  # 'holo4k/121p.pdb' → '121p'
        except IndexError:
            continue

        ligands = parts[1].split(",")  # '3PG,ADP' → ['3PG', 'ADP']
        lig_info[pdb_id] = ligands

    return lig_info


In [7]:
code_path = os.path.abspath("datasets/p2rank-datasets/")

In [8]:
with open(os.path.join(code_path, "holo4k(mlig).ds"), "r") as f:
    lines = f.readlines()

lig_info = preprocessing_lig_code(lines)
print(lig_info["13pk"])  # ['3PG', 'ADP']


['3PG', 'ADP']


In [9]:
print(lig_info["121p"])  # ['GCP']
print(lig_info["182l"])  # ['BZF']

['GCP']
['BZF']


In [10]:
from scipy.spatial import distance_matrix
from multiprocessing import Process, Queue, Pool

In [11]:
from scipy.spatial import distance_matrix
from multiprocessing import Process, Queue, Pool

In [12]:
amino_acids_short = {"ALA":"A", "ARG":"R", "ASN":"N", "ASP":"D", "CYS":"C", "GLU":"E", "GLN":"Q", "GLY":"G", "HIS":"H", "ILE":"I", "LEU":"L", "LYS":"K", "MET":"M", "PHE":"F", "PRO":"P", "SER":"S", "THR":"T", "TRP":"W", "TYR":"Y", "VAL":"V", "SEC":"U", "PYL":"O"}

In [13]:
def parallelize_dataframe(df, func, num_partitions=5):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_partitions)
    results = pool.map(func, df_split)
    pool.close()
    pool.join()
    return results

In [14]:
def get_binding_sites(protein_coords, ligand_coords, protein_atom_residues):
    P_L_distance_matrix = distance_matrix(protein_coords, ligand_coords)
    return sorted(list(set(protein_atom_residues[np.where(P_L_distance_matrix<=4.)[0]])))

In [15]:
from Bio.PDB import PDBParser

pdb_parser = PDBParser(QUIET=True)

def get_info(pdb):
    """ Load protein info """
    structure = pdb_parser.get_structure(pdb, f"{pdb_path}/{pdb}_chimera.pdb")
    
    chain_name_list, pdb_sequence_list, seq_lengths_list, protein_atom_coords, protein_atom_residue_list, reindex = list(), list(), list(), list(), list(), 0
    ligand_total_coords, ligand_chain_list, ligand_chain_number, ligand_chain_code = list(), list(), list(), list()
    
    try:
        lig_code = lig_info[pdb]
    except KeyError:
        return None 
        # 일단 테스트용이라 없으면 넘기기

    """ Exclude complex with 20 amino acids ligand """
    for i in lig_code:
        if i in amino_acids_short.keys():
            return None
        
    """ Extract protein info """
    for chain_name in list(structure[0].child_dict.keys()):
        chain = structure[0][chain_name]

        pdb_sequence, binding_index_list = "", list()
        
        for residue in chain.get_residues():
            if residue.resname in amino_acids_short.keys():
                pdb_sequence += amino_acids_short[residue.resname]
                
                # protein info
                for atom in residue:
                    protein_atom_coords.append(atom.get_coord())
                    protein_atom_residue_list.append(reindex)
                reindex += 1 
                
            elif residue.resname in lig_code:
                ligand_coords = list()
                
                ligand_chain_list.append(chain_name)
                ligand_chain_number.append(str(residue.get_id()[1]))
                ligand_chain_code.append(residue.resname)
                
                # ligand info
                for atom in residue:
                    ligand_coords.append(atom.get_coord())  
                ligand_total_coords.append(ligand_coords)
                
        if len(pdb_sequence) != 0:
            chain_name_list.append(chain_name)
            pdb_sequence_list.append(pdb_sequence)
            seq_lengths_list.append(len(pdb_sequence))                


    """ Get binding sites info """
    for lig_coordi in ligand_total_coords:
        if len(protein_atom_coords) != 0 and len(lig_coordi) !=0:
            binding_index = get_binding_sites(protein_atom_coords, lig_coordi, np.array(protein_atom_residue_list)) 

            binding_index = list(map(str, binding_index))
            binding_index_list.append(",".join(binding_index))

        else:
            print(pdb)
            return None
            
    total_seq_lengths = np.sum(np.array(seq_lengths_list))
    seq_lengths_list = list(map(str, seq_lengths_list))
    
    return ",".join(chain_name_list), ",".join(pdb_sequence_list), total_seq_lengths, ",".join(seq_lengths_list), "|".join(binding_index_list), ",".join(ligand_chain_list), ",".join(ligand_chain_number), ",".join(ligand_chain_code)

In [16]:
def get_raw_protein_info_bulk(df):
    return df.PDB.map(get_info)    

In [17]:
# 1) chimera 파일만 고르기
chimera_files = [
    f for f in complex_list
    if os.path.basename(f).endswith("_chimera.pdb")
]

# 2) 파일명에서 확장자/접미사 제거 → PDB ID로 정규화
cleaned_list = [
    os.path.splitext(os.path.basename(f))[0].replace("_chimera", "")
    for f in chimera_files
]

# (중복 제거 + DF 생성)
data_df = pd.DataFrame({"PDB": cleaned_list}).drop_duplicates("PDB").reset_index(drop=True)
data_df

,PDB
0,1jvg
1,1d7e
2,1f28
3,1e7s
4,1kic
...,...
4538,1b09
4539,1ay5
4540,1dbt
4541,1dxo


In [18]:
info_results = parallelize_dataframe(data_df, get_raw_protein_info_bulk, num_partitions = 5)

In [19]:
info_results = pd.concat(info_results)

In [20]:
data_df["Chain"] = info_results.map(lambda a: a[0] if a is not None else None)
data_df["Sequence"] = info_results.map(lambda a: a[1] if a is not None else None)
data_df["Total_seq_lengths"] = info_results.map(lambda a: a[2] if a is not None else None)
data_df["Chain_seq_lengths"] = info_results.map(lambda a: a[3] if a is not None else None)
data_df["BS"] = info_results.map(lambda a: a[4] if a is not None else None)
data_df["Ligand_chain"] = info_results.map(lambda a: a[5] if a is not None else None)
data_df["Ligand_chain_number"] = info_results.map(lambda a: a[6] if a is not None else None)
data_df["Ligand_chain_code"] = info_results.map(lambda a: a[7] if a is not None else None)
data_df = data_df.loc[data_df.Sequence.isna()==False].reset_index(drop=True)
data_df = data_df.loc[data_df.Chain != " "].reset_index(drop=True)

In [21]:
data_df

,PDB,Chain,Sequence,Total_seq_lengths,Chain_seq_lengths,BS,Ligand_chain,Ligand_chain_number,Ligand_chain_code
0,1jvg,"A,B",NINDLKLTLSKAGQEHLLRFWNELEEAQQVELYAELQAMNFEELNF...,965.0,"485,480","97,98,99,100,154,185,209,211,212,240,241,279,2...","A,B","1901,1902","UD2,UD2"
1,1f28,"A,B,C,D",NAEEQQYLNLVQYIINHGEDRPDRTGTGTLSVFAPSPLKFSLRNKT...,1180.0,"295,295,295,295","23,110,167,170,171,195,196,197,198,199,203,207...","A,A,B,B,C,C,D,D","301,401,302,402,303,403,304,404","UMP,F89,UMP,F89,UMP,F89,UMP,F89"
2,1e7s,A,KQRVFIAGHRGMVGSAIRRQLEQRGDVELVLRTRDELNLLDSRAVH...,314.0,314,"7,9,10,11,12,33,36,37,38,39,59,60,61,63,64,65,...",A,1317,NAP
3,1lpg,"A,B",RKLCSLDNGDCDQFCHEEQNSVVCSCARGYTLADNGKACIPTGPYP...,287.0,"53,234","135,136,137,184,187,188,189,214,231,232,233,23...",B,301,IMA
4,1foa,A,AVIPILVIACDRSTVRRCLDKLLHYRPSAELFPIIVSQDCGHEETA...,342.0,342,"7,8,9,11,38,39,78,80,81,84,85,105,106,107,163,...",A,449,UD1
...,...,...,...,...,...,...,...,...,...
3374,1b09,"A,B,C,D,E",QTDMSRKAFVFPKESDTSYVSLKAPLTKPLKAFTVCLHFYTELSST...,1030.0,"206,206,206,206,206","60,65,73,80,137,138,139,146,149|266,271,286,34...","A,B,C,D,E","500,500,500,500,500","PC,PC,PC,PC,PC"
3375,1ay5,"A,B",MLGNLKPQAPDKILALMGEFRADPRQGKIDLGVGVYKDATGHTPIM...,782.0,"394,388","33,126,179,370,452|64,276,281,406,420,421,514,...","A,B","414,414","MAE,MAE"
3376,1dbt,"A,B,C",MKNNLPIIALDFASAEETLAFLAPFQQEPLFVKVGMELFYQEGPSI...,705.0,"237,234,234","10,32,59,61,121,122,159,181,184,193,211,213,21...","A,B,C","250,251,252","U5P,U5P,U5P"
3377,1dxo,"A,B,C,D",VGRRALIVLAHSERTSFNYAMKEAAAAALKKKGWEVVESDLYAMNF...,1092.0,"273,273,273,273","10,14,15,16,17,19,101,102,103,104,105,146,147,...","A,A,B,B,C,C,D,D","301,302,301,302,301,302,301,302","FAD,DQN,FAD,DQN,FAD,DQN,FAD,DQN"


In [22]:
lengths = data_df.Total_seq_lengths.values

In [23]:
data_df = data_df[lengths <= 1500].reset_index(drop=True)
data_df

,PDB,Chain,Sequence,Total_seq_lengths,Chain_seq_lengths,BS,Ligand_chain,Ligand_chain_number,Ligand_chain_code
0,1jvg,"A,B",NINDLKLTLSKAGQEHLLRFWNELEEAQQVELYAELQAMNFEELNF...,965.0,"485,480","97,98,99,100,154,185,209,211,212,240,241,279,2...","A,B","1901,1902","UD2,UD2"
1,1f28,"A,B,C,D",NAEEQQYLNLVQYIINHGEDRPDRTGTGTLSVFAPSPLKFSLRNKT...,1180.0,"295,295,295,295","23,110,167,170,171,195,196,197,198,199,203,207...","A,A,B,B,C,C,D,D","301,401,302,402,303,403,304,404","UMP,F89,UMP,F89,UMP,F89,UMP,F89"
2,1e7s,A,KQRVFIAGHRGMVGSAIRRQLEQRGDVELVLRTRDELNLLDSRAVH...,314.0,314,"7,9,10,11,12,33,36,37,38,39,59,60,61,63,64,65,...",A,1317,NAP
3,1lpg,"A,B",RKLCSLDNGDCDQFCHEEQNSVVCSCARGYTLADNGKACIPTGPYP...,287.0,"53,234","135,136,137,184,187,188,189,214,231,232,233,23...",B,301,IMA
4,1foa,A,AVIPILVIACDRSTVRRCLDKLLHYRPSAELFPIIVSQDCGHEETA...,342.0,342,"7,8,9,11,38,39,78,80,81,84,85,105,106,107,163,...",A,449,UD1
...,...,...,...,...,...,...,...,...,...
3294,1b09,"A,B,C,D,E",QTDMSRKAFVFPKESDTSYVSLKAPLTKPLKAFTVCLHFYTELSST...,1030.0,"206,206,206,206,206","60,65,73,80,137,138,139,146,149|266,271,286,34...","A,B,C,D,E","500,500,500,500,500","PC,PC,PC,PC,PC"
3295,1ay5,"A,B",MLGNLKPQAPDKILALMGEFRADPRQGKIDLGVGVYKDATGHTPIM...,782.0,"394,388","33,126,179,370,452|64,276,281,406,420,421,514,...","A,B","414,414","MAE,MAE"
3296,1dbt,"A,B,C",MKNNLPIIALDFASAEETLAFLAPFQQEPLFVKVGMELFYQEGPSI...,705.0,"237,234,234","10,32,59,61,121,122,159,181,184,193,211,213,21...","A,B,C","250,251,252","U5P,U5P,U5P"
3297,1dxo,"A,B,C,D",VGRRALIVLAHSERTSFNYAMKEAAAAALKKKGWEVVESDLYAMNF...,1092.0,"273,273,273,273","10,14,15,16,17,19,101,102,103,104,105,146,147,...","A,A,B,B,C,C,D,D","301,302,301,302,301,302,301,302","FAD,DQN,FAD,DQN,FAD,DQN,FAD,DQN"


In [24]:
def fwrite(fw, lines):
    
    for i in lines:
        fw.write("%s\n"%i)
    fw.close()

def extract_ligand_info(lines, chain, number, code):
    return [line.strip() for line in lines if (line[17:20].strip() == code) and (line[21:22].strip() == chain) and (line[22:26].strip() == number)]
    
def add_ligand(row):
    SMILES = list()
    
    coach_PDB, ligand_chain, ligand_chain_number, ligand_chain_code = row.values[0], row.values[6].split(","), row.values[7].split(","), row.values[8].split(",")
    lines = read_file(open(f"{pdb_paths}/{coach_PDB}_chimera.pdb", "r"))
    
    for chain, number, code in zip(ligand_chain, ligand_chain_number, ligand_chain_code):
        ligand_files = extract_ligand_info(lines, chain, number, code)

        fwrite(open(f"{pdb_paths}/{coach_PDB}_{chain}_{number}_{code}_ligand.pdb", "w"), ligand_files)
        
        try:
            command = f"obabel -ipdb {pdb_paths}/{coach_PDB}_{chain}_{number}_{code}_ligand.pdb -osmi -xC | obabel -ismi -osmi -xk -O tmp.smi"

            os.system(command)

            smiles = read_file(open("tmp.smi"))[0].split('\t')[0].strip()            

            smiles = MolToSmiles(MolFromSmiles(smiles),isomericSmiles = False, kekuleSmiles = True)
            SMILES.append(smiles)

        except Exception as e:
            print(coach_PDB, e)
            return None  
    
    return ",".join(SMILES) 

In [ ]:
SMILES = data_df.apply(add_ligand, axis = 1)

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /workspace/datasets/p2rank-datasets/holo4k/1e7s_A_1317_NAP_ligand.pdb)

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
[08:51:37] Explicit valence for atom # 15 N, 4, is greater than permitted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted


1lpg Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(RDKit::ROMol mol, RDKit::SmilesWriteParams params)


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /workspace/datasets/p2rank-datasets/holo4k/1dr5_A_191_NAP_ligand.pdb)

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /workspace/datasets/p2rank-datasets/holo4k/1oqc_A_126_FAD_ligand.pdb)

1 molecule converted
1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /workspace/datasets/p2rank-datasets/holo4k/1oqc_B_126_FAD_ligand.pdb)

1 molecule converted
1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /workspace/datasets/p2rank-datasets/holo4k/1oqc_C_126_FAD_ligand.pdb)

1 molecule converted
1 molecule converted
*** Open Babel Warning

1dwv Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(RDKit::ROMol mol, RDKit::SmilesWriteParams params)


1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
[08:51:38] Explicit valence for atom # 35 N, 4, is greater than permitted


...

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /workspace/datasets/p2rank-datasets/holo4k/4aah_C_701_PQQ_ligand.pdb)

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /workspace/datasets/p2rank-datasets/holo4k/1akv_A_152_FMN_ligand.pdb)

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /workspace/datasets/p2rank-datase

1jdx Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(RDKit::ROMol mol, RDKit::SmilesWriteParams params)


1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /workspace/datasets/p2rank-datasets/holo4k/1cf2_P_340_NAP_ligand.pdb)

1 molecule converted
1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /workspace/datasets/p2rank-datasets/holo4k/1cf2_R_340_NAP_ligand.pdb)

1 molecule converted
1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /workspace/datasets/p2rank-datasets/holo4k/1cf2_O_340_NAP_ligand.pdb)

1 molecule converted
1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /workspac

1o3m Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(RDKit::ROMol mol, RDKit::SmilesWriteParams params)


1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /workspace/datasets/p2rank-datasets/holo4k/1mvn_A_1002_FMN_ligand.pdb)

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecul

In [ ]:
data_df["SMILES"] = SMILES

In [ ]:
data_df = data_df.loc[data_df.SMILES.isna()==False].reset_index(drop=True)
data_df

In [ ]:
def get_SMILES_length(df):
    SMILES, index = df.SMILES.values, list()
    
    for smiles in SMILES:
        for smi in smiles.split(","):
            if len(smi) > 160:
                index.append(False)
                break
        index.append(True)
    
    return index

In [ ]:
smiles_index = get_SMILES_length(data_df)

In [ ]:
mask = pd.Series(smiles_index, dtype=bool)
mask = mask.reindex(data_df.index, fill_value=False)
data_df = data_df[mask.values].reset_index(drop=True)

In [ ]:
data_df

In [ ]:
def read_file(file):
    return file.readlines()

def preprocessing_PDBSWS(lines):
    data = dict()
    
    for line in lines:
        line_list = line.strip().split(" ")
        if len(line_list) == 3:
            data[f"{line_list[0]}_{line_list[1]}"] = line_list[-1]
            
    return data

def preprocessing_SIFTS(lines):
    data = dict()
    
    for line in lines[2:]:
        line_list = line.strip().split("\t")
        data[f"{line_list[0]}_{line_list[1]}"] = line_list[2]
    return data

In [ ]:
SIFTS_mapping_table = preprocessing_SIFTS(read_file(open(f"{info_path}/SIFTS_chain_mapping_table.txt", "r")))
PDBSWS_mapping_table = preprocessing_PDBSWS(read_file(open(f"{info_path}/PDBSWS_chain_mapping_table.txt", "r")))


In [ ]:
def get_uniprot(row):
    try:
        pdb = row.PDB
        chains = row.Chain.split(",")

        uniprot_ids = list()
        for chain in chains:
            name = f"{pdb}_{chain}"
            if name in SIFTS_mapping_table:
                uniprot_ids.append(SIFTS_mapping_table[name])
            else:
                uniprot_ids.append(PDBSWS_mapping_table[name])
        return ",".join(uniprot_ids)    
    
    except Exception as e:
        print(row.PDB, e)
        return None

In [ ]:
def get_uniprot_id_bulk(df):
    return df.apply(get_uniprot, axis=1)

In [ ]:
uniprot_ids = parallelize_dataframe(data_df, get_uniprot_id_bulk, num_partitions = 5)

In [ ]:
data_df["Uniprot_ID"] = pd.concat(uniprot_ids)

In [ ]:
data_df = data_df.loc[data_df.Uniprot_ID.isna()==False].reset_index(drop=True)
data_df

In [ ]:
data_df = data_df.iloc[:, [0, 2, 5]]

In [ ]:
data_df.to_csv("binding_site/datasets/HOLO4K_data.tsv", sep = "\t", index = False)

In [ ]:
%%bash -s "datasets/p2rank-datasets/holo4k"
set -euo pipefail
dir="$1"

# 1) 먼저 어떤 파일이 지워질지 미리보기
find "$dir" -maxdepth 1 -type f -name '*_chimera.pdb.pdb' -print

# 2) 실제 삭제
#find "$dir" -maxdepth 1 -type f -name '*_chimera_chimera.pdb' -delete
#echo "Done."